___
## Delay specific decoder
___

In [ ]:
path_to_data = r'C:\Users\tiffany.ona\Documents\Ephys\summary_complete'
save_data = r'C:\Users\tiffany.ona\Documents\working_memory\data\1_5_Review_Decoder across delays trained on 3s or 10s delays'
save_figures = r'G:\My Drive\WORKING_MEMORY\PAPER\2ND_SUBMISSION_NAT_NEURO\figures_for_resubmission'

In [1]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, RobustScaler

import os 
import pandas as pd
import seaborn as sns
sns.set_context('talk')

import sys
sys.path.append("G:/My Drive/WORKING_MEMORY/EXPERIMENTS/ELECTROPHYSIOLOGY/ANALYSIS/src/functions/")
import ephys_functions as ephys

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings. filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'


**Functions**

In [2]:
def interval_extraction_trial(df, cluster_list=[], variable = 'vector_answer', align = 'Delay_OFF', start = 0, stop = 1, delay_only=False):
    y = []
    d = {}
    
    if delay_only == False:
        # print('Skipping delays')
        if align == 'Delay_OFF' and start < 0:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2)]
        if align == 'Delay_OFF' and start < -1:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2) & (df.delay != 1)]

        if align == 'Stimulus_ON' and stop > 0.5:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2)]

        if align == 'Stimulus_ON' and stop > 1.5:
            df = df.loc[(df.delay != 0.1) & (df.delay != 0.2) & (df.delay != 1)]
    
    # print('Recovered from: ', str(len(df.trial.unique())), ' trials')
    # Create new aligment to the end of the session
    df['a_'+align] = df.fixed_times-df[align]

    # cluster_list = df_all.cluster_id.unique()
    df = df.sort_values('trial')
    
    y = df.groupby('trial')[variable].mean()

    # Filter for the spikes that occur in the interval we are analyzing
    df = df.loc[(df['a_'+align]>start)&(df['a_'+align]<stop)]

    df_final = pd.DataFrame()
    df_final = df.groupby(['trial','cluster_id']).count()
    df_final.reset_index(inplace=True)
    df_final = df_final.pivot_table(index=['trial'], columns='cluster_id', values='fixed_times', fill_value=0).rename_axis(None, axis=1)
    df_final = df_final.reindex(cluster_list, axis=1,fill_value=0)

    result = pd.merge(df_final, y, how="right", on=["trial"]).fillna(0)
    result = result.rename(columns={variable: "y"})
    result['y'] = np.where(result['y'] == 0, -1, result['y']) 
    
    return result, result['y']

In [3]:
def train(df, decode='vector_answer', align='Delay_OFF', start=-0.5, stop=0, cluster_list = [], ratio=0.65, test_index=[],  train_index=[], fakey=[], delay_only=False):

    df_final, y = interval_extraction_trial(df,variable = decode, align = 'Stimulus_ON', start = -0.25, stop = 0, cluster_list = cluster_list, delay_only=delay_only)
    
    sc = RobustScaler()
    x = df_final.iloc[:, df_final.columns != 'y']
    sc_fit = sc.fit(x)
    
    df_final, y = interval_extraction_trial(df,variable = decode, align = align, start = start, stop = stop, cluster_list = cluster_list, delay_only=delay_only)
    
    # This is mainly for the session shuffles
    if len(fakey) > 1:
        print('Using shuffled session')
        y = fakey[len(fakey)-len(y):]
        df_final['y'] = y   
        
    train_cols = df_final.columns
    
    #Train the model   
    df_final.reset_index(inplace=True)
    df_final = df_final.drop(columns ='trial')
    
    if len(test_index) >= 1:
        print('Using splits')
        train = df_final.loc[train_index,:]
        test = df_final.loc[test_index,:]
        # print('Fold',str(fold_no),'Class Ratio:',sum(test['y'])/len(test['y']))
        x_test = test.iloc[:, test.columns != 'y']
        y_test = test['y']
        x_train = train.iloc[:, train.columns != 'y']
        y_train = train['y']
        
    else:
        x_train = df_final.iloc[:, df_final.columns != 'y']
        y_train = df_final['y']
        x_test = x_train
        y_test = y_train
        
    #Normalize the X data
    # sc = RobustScaler()
    # sc_fit = sc.fit(df_final.iloc[:, df_final.columns != 'y'])
    
    # x_train = sc.fit_transform(x_train)
    # x_test = sc.fit_transform(x_test)
    
    x_train = sc_fit.transform(x_train)
    x_test = sc_fit.transform(x_test)

    model = LogisticRegression(solver='liblinear', class_weight='balanced', penalty = 'l1').fit(x_train, y_train)
    # model = LogisticRegression(solver='liblinear', penalty = 'l1', C=0.95).fit(x_train, y_train)
    train_cols = df_final.columns
    
    p_pred = model.predict_proba(x_test)    
    y_pred = model.predict(x_test)    
    f1score= f1_score(y_test, y_pred, average='weighted')

    y_test = np.where(y_test == -1, 0, y_test) 
    y_new = y_test.reshape(len(y_test), 1).astype(int)
    score_ =  np.take_along_axis(p_pred,y_new,axis=1)   

    # print('Trained model on ', len(train_cols), ' neurons.')
    print('score:', np.mean(score_), 'f1_score ', f1score)
    
    return model, train_cols, np.mean(score_), sc_fit

In [4]:
def test(df,sc_fit, epoch='Stimulus_ON',initrange=-0.4,endrange=1.5,r=0.2, model = None, train_cols=None, variable='ra_accuracy',
                      hit=1, nsurrogates = 100, decode='vector_answer', ratio=0, cluster_list = [], test_index=[], fakey=[], delay_only=False):
    '''
    Function that tests a previously trained function (func. train_decoder) on population activity of specific segments
    
    Attributes
        - df: DataFrame. it contains a whole ephys session without curation. 
        - WM and RL are the variables to consider a trial in the RL or in the WM-module. Both need to be floats. 
        - epoch: str. Moment at which the data will be aligned to. 
        - initrange: float. 
        - endrange: float.
        - r: float 
        - model. function. 
        - train_cols
        - name. String
        - variables. List. 
        - hits. List. 
        - colors. List
        - nsurrogates. Int. 
        - indexes. List 
        - decode. String
    
    Return
        - df_real
        - df_iter
        It will also make a plot. 
    '''
    
    df_real = pd.DataFrame()
    df_iter = pd.DataFrame(columns=['iteration', 'score', 'times', 'epoch', 'variable'])
        
    times = [] # Timestamps
    real_score = [] # real scoring of the decoded
    mean_sur=[] # mean of the surrogate data

    for start, stop in zip(np.arange(initrange,endrange-r,r/2),np.arange(initrange+r,endrange,r/2)):
        times.append((start+stop)/2)
        df_final, y = interval_extraction_trial(df,variable = decode, align = epoch, start = start, stop = stop, cluster_list=cluster_list, delay_only=delay_only)

        # Sometimes the testing and the trainind dataset have different neurons since they are looking at different trials and perhaps there were no spikes
        # coming from all neurons. We compare which columns are missing and add them containing 0 for the model to work. 
        test_cols = df_final.columns
        common_cols = train_cols.intersection(test_cols)
        train_not_test = train_cols.difference(test_cols)
        for col in train_not_test:
            df_final[col] = 0

        #The other way round. When training in segmented data, sometimes the training set is smaller than the testing (for instance, when training in Hb trials and testing in WM)
        test_not_train = test_cols.difference(train_cols)
        for col in test_not_train:
            df_final.drop(columns=[col],inplace=True)

        #Train the model"
        if len(test_index) >= 1:
            print('Train splitting trials')
            # Split data in training and testing
            # x_train, x_test, y_train, y_test =\
            #     train_test_split(df_final, y, test_size=test_sample,random_state=random_state)
            
            df_final.reset_index(inplace=True)
            df_final = df_final.drop(columns ='trial')
            test = df_final.loc[test_index,:]
            # print('Fold',str(fold_no),'Class Ratio:',sum(test['y'])/len(test['y']))
            x_test = test.iloc[:, test.columns != 'y']
            y_test = test['y']             

        else:
            x_train = df_final.iloc[:, df_final.columns != 'y']
            y_train = df_final['y']
            x_test = x_train
            y_test = y_train
        
        #Normalize the X data
        # sc = RobustScaler()
        # x_test = sc.fit_transform(x_test)
        # x_test = sc_fit.transform(x_test)
        
        p_pred = model.predict_proba(x_test)
        y_pred = model.predict(x_test)
        score_ = model.score(x_test, y_test)
        real_score.append(score_)

        # y_test = np.where(y_test == -1, 0, y_test) 
        # y_new = y_test.reshape(len(y_test), 1).astype(int)
        # corrected_score =  np.take_along_axis(p_pred,y_new,axis=1)   
        # real_score.append(np.mean(corrected_score))

        # print('score:', score_, 'corrected score: ', np.mean(corrected_score), end='\n\n')

        i=0
        rows = []
        while i <= nsurrogates:
            i+=1
            y_perr = shuffle(y_test)
            score_ = model.score(x_test, y_perr)

            # y_new = y_perr.reshape(len(y_perr), 1).astype(int)
            # result =  np.take_along_axis(p_pred,y_new,axis=1)     
            # score_  = np.mean(result)

            new_row = pd.DataFrame({'iteration': [i], 'score': [score_], 'times': [(start+stop)/2], 'epoch' : [epoch], 
                                      'variable' : [variable+'_'+str(hit)]})
            df_iter = pd.concat([df_iter,new_row], ignore_index=True)
        
    times.append('trial_type')
    real_score.append(variable+'_'+str(hit))
    a_series = pd.DataFrame([real_score], columns = times)

    df_real = pd.concat([df_real,a_series], ignore_index=True)
    
    return df_real, df_iter

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

os.chdir(path_to_data)
for filename in os.listdir(os.getcwd()):
# for filename in list_of_sessions:
    # 
    if filename[-3:] != 'pdf':
        df = pd.read_csv(filename, sep=',',index_col=0)
    else:
        continue
        
    print(filename, '/ Total session trials: ', len(df.trial.unique()), '/ Number of neurons: ', len(df.cluster_id.unique()))
    
    df = ephys.add_time_before_stimulus(df, 4)

    substract = True
    df['delay'] = np.around(df.delay,2)
    # Variables used for decoder training
    decode = 'vector_answer'
    align='Delay_OFF'
    ratio = 0.6
    delay_train = 'all'
    start = -0.5
    stop = 0
    type_trial ='WM_roll'
    hit = 1
    nsplits = 5
    
    cluster_list = df.cluster_id.unique()

    skf = StratifiedKFold(n_splits=nsplits, shuffle=True)
    # skf = KFold(n_splits=nsplits, shuffle=True)

    # Create a dataframe for training data
    if type_trial == 'all':
        df_train = df.loc[(df.hit==hit)]
    elif hit == 'all':
        df_train = df.loc[(df[type_trial]>=ratio)]
    else:
        df_train = df.loc[(df[type_trial]>=ratio)&(df.hit==hit)]

    if delay_train != 'all':
        df_train = df_train.loc[(df_train.delay==delay_train)]
    else:
        df_train = df_train.loc[(df_train.delay!=0.2)&(df_train.delay!=0.1)]
        
    df_final, y = interval_extraction_trial(df_train, variable = decode, align = align, start = start, stop = stop, cluster_list=cluster_list)
    df_final.reset_index(inplace=True)
    df_final = df_final.drop(columns ='trial')
    df_final = df_final.drop(columns ='y')

    # Assuming df is your DataFrame with neural spike data
    # Replace 'features' and 'target' with the actual column names
    X = df_final  # Features (neural spike data)
    y = y  # Target variable

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define a pipeline for scaling and logistic regression
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardize features
        ('logreg', LogisticRegression(max_iter=500))  # Logistic Regression
    ])

    # Define the parameter grid
    param_grid = {
        'logreg__solver': ['liblinear', 'saga'],  # Solvers to test
        'logreg__penalty': ['l1', 'l2'],          # Regularization types
        'logreg__C': [0.01, 0.1, 1, 10, 100]     # Regularization strength
    }

    # Set up GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

    # Fit the grid search to the training data
    grid_search.fit(X_train, y_train)

    # Print the best parameters and corresponding score
    print("Best Parameters:", grid_search.best_params_)
    print("Best Cross-Validated Accuracy:", grid_search.best_score_)

    # Evaluate the model on the test set
    best_model = grid_search.best_estimator_
    test_score = best_model.score(X_test, y_test)
    print("Test Set Accuracy:", test_score)


NameError: name 'path_to_data' is not defined

_________________________________________
#### **Train in one delay and test in other delays**

In [ ]:
## Dataframe used for cumulative analysis
df_cum = pd.DataFrame()
df_cum_shuffle = pd.DataFrame()

os.chdir(path_to_data)
for filename in os.listdir(os.getcwd()):
    if filename[-3:] != 'pdf':
        df = pd.read_csv(filename, sep=',',index_col=0)
    else:
        continue
        
    print(filename, '/ Total session trials: ', len(df.trial.unique()), '/ Number of neurons: ', len(df.cluster_id.unique()))
    
    df = ephys.add_time_before_stimulus(df, 4)

    substract = True
    df['delay'] = np.around(df.delay,2)
    
    if 10 not in df.delay.unique():
        print('Skip session because no 10s delay')
        continue
    
    # Variables used for decoder training
    decode = 'vector_answer'
    align = 'Delay_OFF'
    ratio = 0.6
    delay_train = 'all'
    start = -0.5
    stop = 0
    type_trial ='WM_roll'
    hit = 1
    nsplits = 3
    
    #Variables for testing
    colors=['darkgreen','darkorange', 'crimson', 'pink']
    variables = ['WM_roll','WM_roll','RL_roll','RL_roll']
    hits = [1,1,1,1]
    ratios = [0.6,0.6,0.4, 0.4]
    delays = [3, 10, 3, 10]
    variables_combined = [f"{variables[i]}_{delays[i]}" for i in range(len(variables))]
            
    cluster_list = df.cluster_id.unique()

    skf = StratifiedKFold(n_splits=nsplits, shuffle=True)

    # Create a dataframe for training data
    if type_trial == 'all':
        df_train = df.loc[(df.hit==hit)]
    elif hit == 'all':
        df_train = df.loc[(df[type_trial]>=ratio)]
    else:
        df_train = df.loc[(df[type_trial]>=ratio)&(df.hit==hit)]
    
    if delay_train != 'all':
        df_train = df_train.loc[(df_train.delay==delay_train)]
    else:
        df_train = df_train.loc[(df_train.delay!=0.2)&(df_train.delay!=0.1)]
            
    df_final, y = interval_extraction_trial(df_train, variable = decode, align = align, start = start, stop = stop, cluster_list=cluster_list)
    df_final.reset_index(inplace=True)
    df_final = df_final.drop(columns ='trial')

    fold_no = 1
    if len(y) < nsplits:
        print('Skip session because not enough trials')
        continue
        
    for train_index, test_index in skf.split(df_final, y):
        model, train_cols, score, sc_fit = train(df_train, decode=decode, align=align, start=start,stop=stop, cluster_list = cluster_list, 
                                  test_index=test_index,  train_index=train_index)

        # Remove a fifth of the dataset so it can be compared to crossvalidated data. If we want to randomly reduce it, add reduce to trian function
        # drop_list = np.array_split(df_train.trial.unique(), 5)[fold_no]
        # df_train = df_train[~df_train['trial'].isin(drop_list)]
        # index_train_trials = df_train.trial.unique()
        # print('Total of left: ', len(df_train.loc[df_train['vector_answer'] == 0].groupby('trial').mean()), '; Total of right: ', len(df_train.loc[df_train['vector_answer'] == 1].groupby('trial').mean()))

        # fig, ax1 = plt.subplots(1,1, figsize=(8, 4), sharey=True)
        for color, variable, delay, hit, ratio in zip(colors,variables, delays, hits,ratios):
            df_res = pd.DataFrame()
            df_sti = pd.DataFrame()
            df_iter = pd.DataFrame()
            try:
                df_delay = df.loc[np.around(df.delay,1)==delay]
                delay=np.around(df_delay.delay.iloc[0],1)
                # print('Delay:', delay)
            except:
                pass

            if delay == 0.1 or delay == 0.2:
                endrange=3.5
                r=0.25

            elif delay == 1:
                endrange=4.5
                r=0.25

            elif delay == 3:
                endrange=6.5    
                r=0.25
                
            elif delay == 10:
                endrange=14.5
                r=0.25
            
            # Create a dataframe for testing data
            if variable == 'all':
                df_test = df_delay.loc[(df_delay.hit==hit)]
            elif hit == 'all':
                df_test = df_delay.loc[(df_delay[variable]>=ratio)]
            else:
                df_test = df_delay.loc[(df_delay[variable]>=ratio)&(df_delay.hit==hit)]
                
            if fold_no == 1:
                print(delay, variable, 'Threshold:', ratio, 'Hit:', hit, 'Nº of trials:', len(df_test.trial.unique()))

# -----------  Remove the trials that overlap with the training set.
            list_train_trials = df_train.trial.unique()[train_index]
            df_test = df_test[~df_test['trial'].isin(list_train_trials)] 

            if len(df_test.trial.unique()) < 5:
                print('Not enough trials with this condition')
                continue
            
            if delay_train != delay:
                # print('Reduced trials from:', len(df_test.trial.unique()))
                values = df_test.groupby('trial').mean().reset_index()
                select_trials = values.sample(n=len(test_index), replace=True).trial.unique()
                df_test = df_test[df_test['trial'].isin(select_trials)]
                # print('Reduced trials to:', len(df_test.trial.unique()))
                
            if len(df_test.trial.unique()) < 5:
                print('Not enough trials with this condition')
                continue

            df_real,df_temp = test(df_test, sc_fit, decode= decode,epoch='Stimulus_ON',initrange=-2,endrange=endrange, r=r, model = model, delay_only=delay, variable=variable, hit=hit, nsurrogates = 100,train_cols = train_cols, cluster_list = cluster_list)

            df_sti = pd.concat([df_real,df_sti])
            df_iter = pd.concat([df_iter,df_temp])
            
            variable = str(variable)+'_'+str(hit)

            # Aligmnent for Stimulus cue
            real = df_sti.loc[(df_sti['trial_type'] ==variable)].to_numpy()
            times = np.around(np.array(df_sti.columns)[:-1].astype(float),2)

            df_new= df_iter.loc[(df_iter.epoch=='Stimulus_ON')].groupby('times')['score']
            y_mean= df_new.mean().values
            lower =  df_new.quantile(q=0.975, interpolation='linear') - y_mean
            upper =  df_new.quantile(q=0.025, interpolation='linear') - y_mean
            x=times
            
            # ephys.plot_results_decoder(fig, real[0][:len(y_mean)], times, df_new,  ax1, color = color, epoch = 'Delay_OFF', 
            #             y_range = [-0.05, 0.5], x_range = None, substract=True)
            
            df_cum, df_cum_shuffle = ephys.add_to_summary(real[0][:len(y_mean)], y_mean, times, filename, variable, epoch = 'Stimulus_ON', fold_no=fold_no, df_iter=df_iter, df_cum=df_cum, df_cum_shuffle=df_cum_shuffle, substract=True, delay=delay)
            
            sns.despine()
            
        fold_no+=1
        plt.tight_layout()
        plt.show()

E04_2021-03-30_11-20-16.csv / Total session trials:  191 / Number of neurons:  11
Using splits
score: 0.7681217668818022 f1_score  0.793939393939394
3.0 WM_roll Threshold: 0.6 Hit: 1 Nº of trials: 43
10.0 WM_roll Threshold: 0.6 Hit: 1 Nº of trials: 26
3.0 RL_roll Threshold: 0.4 Hit: 1 Nº of trials: 1
Not enough trials with this condition
10.0 RL_roll Threshold: 0.4 Hit: 1 Nº of trials: 1
Not enough trials with this condition


<Figure size 640x480 with 0 Axes>

Using splits
score: 0.6552788564196513 f1_score  0.6343434343434342
Not enough trials with this condition
Not enough trials with this condition


<Figure size 640x480 with 0 Axes>

Using splits
score: 0.7429853630658666 f1_score  0.8785650623885918
Not enough trials with this condition
Not enough trials with this condition


...

Using splits
score: 0.6052219192333103 f1_score  0.6365217391304349
Not enough trials with this condition
Not enough trials with this condition


<Figure size 640x480 with 0 Axes>

Using splits
score: 0.5974791789557176 f1_score  0.6666666666666666
Not enough trials with this condition
Not enough trials with this condition


<Figure size 640x480 with 0 Axes>

AttributeError: 'DataFrame' object has no attribute 'trial'

In [ ]:
df_test.groupby('trial').mean().reset_index()

In [ ]:
# save_path = 'G:/My Drive/WORKING_MEMORY/PAPER/ANALYSIS_figures/'
# os.chdir(save_path)
# df_cum['trial_type'] = np.where(df_cum['delay'] == 3, 'WM_3', df_cum['trial_type'])
# df_cum['trial_type'] = np.where(df_cum['delay'] == 10, 'WM_10', df_cum['trial_type'])
# df_cum['trial_type'] = np.where(df_cum['delay'] == 1, 'WM_1', df_cum['trial_type'])

file_name = '5C_split_delay.csv'
df_cum.to_csv(save_data+file_name)

In [ ]:
def plot_results_session_summary(plot, df, colors, variables_combined = ['WM_roll_1', 'RL_roll_1'], 
                                 y_range = [], x_range = None, epoch = 'Stimulus_ON', baseline=0.5):
    
    for color, variable, ax in zip(colors, variables_combined, np.repeat(plot, len(variables_combined))):
        try:
            df_loop = df.loc[(df['trial_type'] == variable)]
        except:
            df_loop = df

        df_loop = df_loop.dropna(axis=1, how='all')

        # Select only columns where the column name is a number or can be transformed to a number
        numeric_columns = df_loop.columns[df_loop.columns.to_series().apply(pd.to_numeric, errors='coerce').notna()]

        real = np.mean(df_loop.groupby('session')[numeric_columns].mean(), axis=0).to_numpy()
        times = df_loop[numeric_columns].columns.astype(float)
        
        df_results = pd.DataFrame()
        df_results['times'] = times
        df_results['real'] = real
        df_results = df_results.sort_values(by='times')

        if x_range == None:
            x_range = [min(times), max(times)]
            
        mean_surr = []
        df_lower = pd.DataFrame()
        df_upper = pd.DataFrame()

        df_for_boots = df_loop.groupby('session')
        for timepoint in df_results['times'].values:
            mean_surr = []

            # recover the values for that specific timepoint
            try:
                array = df_for_boots[timepoint].mean().to_numpy()
            except:
                array = df_for_boots[str(timepoint)].mean().to_numpy()

            # iterate several times with resampling: chose X time among the same list of values
            for iteration in range(1000):
                x = np.random.choice(array, size=len(array), replace=True)
                # recover the mean of that new distribution
                mean_surr.append(np.mean(x))

            df_lower.at[0, timepoint] = np.percentile(mean_surr, 2.5)
            df_upper.at[0, timepoint] = np.percentile(mean_surr, 97.5)

        lower =  df_lower.iloc[0].values
        upper =  df_upper.iloc[0].values

        ax.plot(df_results.times,df_results.real, color=color)
        ax.fill_between(df_results.times, lower, upper, alpha=0.2, color=color)
        ax.axhline(y=baseline,linestyle=':',color='black')
        ax.set_ylim(y_range)
        ax.set_xlim(x_range)

        if epoch == 'Stimulus_ON':
            ax.set_xlabel('Time to stimulus onset (s)')
            ax.fill_betweenx(np.arange(-1,1.15,0.1), 0,0.4, color='grey', alpha=.4)
        else:
            ax.set_xlabel('Time to go cue (s)')
            ax.fill_betweenx(np.arange(-1,1.15,0.1), 0,0.2, color='grey', alpha=.4)

        sns.despine()

In [ ]:
def plot_results_session_summary_substract(fig, plot, df: pd.DataFrame, df_shuffle: pd.DataFrame, color, variable= 'WM_roll_1', 
                                 y_range = [-0.05, 0.3], x_range = None, epoch = 'Stimulus_ON', baseline=0.5):
    
            # Select only columns where the column name is a number or can be transformed to a number
        numeric_columns = df.columns[df.columns.to_series().apply(pd.to_numeric, errors='coerce').notna()]
        
        df_loop = (df.loc[(df['trial_type'] == variable)].groupby('session').mean()[numeric_columns]
                    - df_shuffle.loc[(df['trial_type'] == variable)].groupby('session').mean()[numeric_columns])

        real = np.array(np.mean(df_loop.groupby('session').mean()[numeric_columns])) 
        times = np.array(np.mean(df_loop[numeric_columns]).index).astype(float)

        df_results = pd.DataFrame()
        df_results['times'] = times
        df_results['real'] = real
        df_results = df_results.sort_values(by='times')

        if x_range == None:
            x_range = [min(times), max(times)]
            
        mean_surr = []
        df_lower = pd.DataFrame()
        df_upper = pd.DataFrame()

        df_for_boots = df_loop.groupby('session').mean()
        for timepoint in times:
            mean_surr = []

            # recover the values for that specific timepoint
            try:
                array = df_for_boots[timepoint].to_numpy()
            except:
                array = df_for_boots[str(timepoint)].to_numpy()

            # iterate several times with resampling: chose X time among the same list of values
            for iteration in range(1000):
                x = np.random.choice(array, size=len(array), replace=True)
                # recover the mean of that new distribution
                mean_surr.append(np.mean(x))

            df_lower.at[0, timepoint] = np.percentile(mean_surr, 2.5)
            df_upper.at[0, timepoint] = np.percentile(mean_surr, 97.5)

        lower =  df_lower.iloc[0].values
        upper =  df_upper.iloc[0].values

        plot.plot(df_results.times,df_results.real, color=color)
        plot.fill_between(df_results.times, lower, upper, alpha=0.2, color=color)
        plot.axhline(y=baseline,linestyle=':',color='black')
        plot.set_ylim(y_range)
        plot.set_xlim(x_range)

        if epoch == 'Stimulus_ON':
            plot.set_xlabel('Time to stimulus onset (s)')
            plot.fill_betweenx(np.arange(-1,1.15,0.1), 0,0.4, color='grey', alpha=.4)
        else:
            plot.set_xlabel('Time to go cue (s)')
            plot.fill_betweenx(np.arange(-1,1.15,0.1), 0,0.2, color='grey', alpha=.4)

        sns.despine()


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 4), sharey=True)
baseline = 0
plot_results_session_summary(ax, df_cum.loc[df_cum.delay == 3], colors, variables_combined = ['WM_roll_1', 'RL_roll_all'], 
                                y_range = [baseline-0.05, baseline+0.3], x_range = [-2,13], epoch = 'Stimulus_ON', baseline=baseline)
# fig.savefig(save_figures+f'/review_decoder_across_delays_train_{train_delay}s.pdf', bbox_inches='tight')

In [ ]:
df_cum = pd.DataFrame()
df_cum_shuffle = pd.DataFrame()

os.chdir(path_to_data)

for filename in os.listdir():
    if filename.endswith('.pdf'):
        continue
    
    df = pd.read_csv(filename, sep=',', index_col=0)
    print(f"{filename} / Total session trials: {df.trial.nunique()} / Number of neurons: {df.cluster_id.nunique()}")
    
    df = ephys.add_time_before_stimulus(df, 4)
    df['delay'] = df['delay'].round(2)
    
    if 10 not in df['delay'].unique():
        print('Skip session because no 10s delay')
        continue
    
    # Variables for decoder training
    decode, align = 'vector_answer', 'Delay_OFF'
    ratio, delay_train, start, stop = 0.6, 'all', -0.5, 0
    type_trial, hit, nsplits = 'WM_roll', 1, 3
    
    # Variables for testing
    test_params = [
        ('WM_roll', 3, 1, 0.6, 'darkgreen'),
        ('WM_roll', 10, 1, 0.6, 'darkorange'),
        ('RL_roll', 3, 'all', 0.4, 'crimson'),
        ('RL_roll', 10, 'all', 0.4, 'pink')
    ]
    
    cluster_list = df.cluster_id.unique()
    skf = StratifiedKFold(n_splits=nsplits, shuffle=True)
    
    df_train = df[df[type_trial] >= ratio] if hit == 'all' else df[(df[type_trial] >= ratio) & (df.hit == hit)]
    
    if delay_train != 'all':
        df_train = df_train[df_train.delay == delay_train]
    else:
        df_train = df_train[~df_train.delay.isin([0.2, 0.1])]
    
    df_final, y = interval_extraction_trial(df_train, variable=decode, align=align, start=start, stop=stop, cluster_list=cluster_list)
    df_final.reset_index(inplace=True)
    df_final.drop(columns='trial', inplace=True)
    
    if len(y) < nsplits:
        print('Skip session because not enough trials')
        continue
    
    fold_no = 1
    for train_idx, test_idx in skf.split(df_final, y):
        model, train_cols, score, sc_fit = train(df_train, decode, align, start, stop, cluster_list, test_idx, train_idx)
        
        for variable, delay, hit, ratio, color in test_params:
            df_delay = df[df.delay.round(1) == delay]
            if df_delay.empty:
                continue
            
            delay = df_delay.delay.iloc[0].round(1)
            endrange, r = {0.1: (3.5, 0.25), 0.2: (3.5, 0.25), 1: (4.5, 0.25), 3: (6.5, 0.25), 10: (14.5, 0.25)}.get(delay, (None, None))
            if endrange is None:
                continue
            
            df_test = df_delay[df_delay[variable] >= ratio] if hit == 'all' else df_delay[(df_delay[variable] >= ratio) & (df_delay.hit == hit)]
            
            if fold_no == 1:
                print(f"{delay} {variable} Threshold: {ratio} Hit: {hit} Nº of trials: {df_test.trial.nunique()}")
            
            df_test = df_test[~df_test['trial'].isin(df_train.trial.unique()[train_idx])]
            
            if delay_train != delay:
                select_trials = df_test.groupby('trial').mean().sample(n=len(test_idx), replace=True).index
                df_test = df_test[df_test['trial'].isin(select_trials)]
                
            if df_test.trial.nunique() < 5:
                print('Not enough trials with this condition')
                continue
            
            df_real, df_temp = test(df_test, sc_fit, decode, 'Stimulus_ON', -2, endrange, r, model, delay, variable, hit, 100, train_cols, cluster_list)
            
            df_cum, df_cum_shuffle = ephys.add_to_summary(df_real.iloc[0, :len(df_temp)], df_temp.groupby('times')['score'].mean().values,
                                                           np.around(df_temp.columns[:-1].astype(float), 2), filename, variable, 'Stimulus_ON',
                                                           fold_no, df_temp, df_cum, df_cum_shuffle, True, delay)
            
            sns.despine()
            
        fold_no += 1
        plt.tight_layout()
        plt.show()
